<a href="https://colab.research.google.com/github/T-Sunm/Learn-pandas/blob/main/Exercise_Grouping_and_Sorting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pitchfork-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F655%2F1252%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240716%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240716T122438Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da6c95dff32861dece437ef29aee42d3baccce34553fb9d0d48ebf9ca2c0bc9fd681e4bb19a719a844fdd34ea1ec53ad4af7b3a4c353a0f585e81735686a15f3be152d09820529f446067b32d194545673eaacf8ed761b0dab4f6f021f2858070663e8ff9adaf28d9a9fb67ab4efaee86651e9f963253b6214f4de46bc37a79e408e7ab08821cbb89973df7c8b21510b32a35d68b1c438ab30fb3b11f62972015a879b87e67cbec6e2dac28603821ef7c9f67e0957c7c1b31f7e582637a609250862379f431fb42a5a71c8ec9ebc71aed315051a80cb60054891676c7f2a94870ac8177d1958b409a4114ba45aef8da2a9aa71ef736d7a10f62cd02f6b420418c,chess:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2321%2F3919%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240716%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240716T122438Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D74d1825c5a2ebc514a970ab0300be69110508c1775df5e79598d93fb080819d1beea713895988e288aaa8eb4e97a04199d45a895fb28042d1496c15cd7144ab0ffba20ac8eafb3616c11f998c4ad13e5e9bd4cb4cc06819f53bd81c2658cdccb619a86818ed326d0db14705a44a94af107abb08dc0a098fd9002d8ed69434c3ae156d72b304dee8378400c75c5d5c8d3bb608b821890bfbee6ecc43738056df5fe50e40fcbd5a333d58f03568c03a163c75f2b240df452e8caa2795bcb6fd7d33a165999524ae2e0fdb515963d9b3f1e84e98ed32ef3658b2ed3efebb39f9dfdb4e61fb9e6d6b1d8990711a08c167a05aba09bedd5d186ec9f05916a7f367a7c,kepler-exoplanet-search-results:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2894%2F4877%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240716%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240716T122438Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da9dec4e207d6c081c073afa86c1294e1707244b24b020cf0639662e391867f89f656e49c4f1fae53d2c36287fbf8185cc96fbac4ef17da449a3767b0074f48cdc64e7c08e759e0a6ccc6d49a32c276bd91322ac37c0293a77a1896b1ec833da8ae912fced8c11618d7900b2dd0cdfcfdf89dc3335180357e83dd7f90e9628f0aa113de2da63e6304b09aeb5021baff41caa09be4bbbf99e8e33d05413183d5dbce9ee7aefbcdaddabad56858be1a58f2b4a2d7093f04cfa1624508bf17b78ea46ab64ab8d7966361fd2e4ecc41afb95b73ec66510af3628799b1ff8d58e1176d56b8eaad430cdd88cff993a12adfaefab6c822bcab8f7e6753075fc47a47c669,things-on-reddit:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3491%2F5624%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240716%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240716T122438Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9dbc3299f1b6f8ce024f7cf19fadb14771d9de5173125820f7e5b44867abe21ae83edd20e414747ea53ee482757bb0d631081cb27329edda86743fc65f45bf985fb4629b52c95586390f4caea723b2ee4709030563a18968422edba5985d901795afa564ae116c49c67a19eab989efb214221d59792c456b19065d6c751c2701597b737a111b65d76d4c12a32ed2a0a8af3dff3c1112a160dc7efb5b5c11ffef142fe370b6992d17f64f449350a6a0e4597e9e9becba7d8b02974b3d9d0a33cc3572b770f25f86348eeff7a51040140aea91552b3fb8855726d49f4fab3b561f12e8b77a968748e0cdaa40be76ed4739ef98b4329307d06a86b7be4071360e33,wine-reviews:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1442%2F8172%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240716%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240716T122438Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8fb0692e5ff55f6cd9d807002d446653e7b0bfb725fca1f3e78bf6928044ac48f75eaba998c589bb7a31c9554396601275322fabbb6b75f158e5a6a762cc8c53bf4dbbdc0641aec5eb1cd1ae595c8cd556ec2f8d6414d7670c51d23b110b0382d7173e3fa0264c70eb97b150125bf3648d39861976b034eb6dfe1d534b0de8794b5158240a77f4541fe9b4fc8688b1681772e7ba883ce83a9d20788ce66d7dbbcb1b7475bd0d58ea9ae2210d2142daac0f342b6d2d95a9fc2435bc1c730554dabc8dc104507c9e45100c6e25a23b0daf71d580e26906d2bf532c817f04c0ade0ff51b684d3317e4ee4ecd3dcff3ff5e85e5cbf7f026b0b2c5a81d23570cfba60,ramen-ratings:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F9366%2F13206%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240716%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240716T122438Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7c8bad66f8e6959c97f11b774a88b6c4d8363b75baedf2baf9af63f076c5f51f38615b03921b11acf193d32112d7baf8c6981d066ccc6a6c300b4add1561fa0051170fd366b99acaea7f2b6fcd8d77cd71a0eb81cd2bdbc2895de87dc5895eeae14c1d9d77c963ada753cee0edff3a8c0191d550c8bb3dee1a348a65b440d78f80607c678c49b80cddf897ab082eac84d2c7f77b9453f9148fce88439ad4bf888b169780e763540618ea01c71e834fbf279309f83bb98b6f46fe3699de26bf23f530072dc4dddc4ee1f76d8f489a300eafc2f01778d407592bf9411cff65e76ce204b8f2f51707591b03f4d4063e0f90b3a5a9df09767797209835aeadd4b16d,powerlifting-database:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F179555%2F403916%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240716%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240716T122438Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8aec27ca20233ee696c71cab49bca349bed3fe57b8e6c3cd62d1db20add13b0ca9499cc53e27b03631dc24528768d3eba23e13f67423539622fc745998f0fa4522083bf66e7df866deaffb2201000aeaae7fb8eff71604d113d64ff52111e521aa53fc7488a94dea1627bff73f548c070f347fcb77ef384da30b9db14c878122485eb396fac2d864a5768c12324ae51382ea64106aa965d62d1d2050983b59e41f0023b4192cf07e1d1f80f893275581a15b48f0b34c75f5ebb16b8732746b39e8c866b30b5fdb689f9c5ff70b1d6a04f2e95f76a338e5e8f05da8490fa4bfc99da66c5cb9416bf0507d65543b51c5f5f2cc6afe1ae093103964c85420530959,youtube-new:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4549%2F466349%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240716%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240716T122438Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9d8bd6f35986f8eeaa9d3a33f70f2f7a42190fb4809b0b84615cfb1a3aabd495b4ddf7a5e6094663128bad050dfba26e70c4cb140a65dba3be1a60a848f713dfabcb34116149256ff3d9ae5c0e36feeb6d4d6c6c768357d3ad66084e4236319a4e6801b7b8107c05f94b6fc9d510d926342a2760b0bd1f1ada6a454f5e217e892dbc3364aebc1b7d55e08a0d9458c9b70650754669d1bdbe85fe30fab3d0de138831ee2b5c7b8605e41b9419567cc60101904e660353f492992816c229df1c17a5db1c092b1a00a1ee9cd61f90acc5cc81aa23337ec92373c8398c0343d01c7b1add2f35650ff800f876492a08a02fe7a6e7905f4327700407f7606ba745be5c,188-million-us-wildfires:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2478%2F1151655%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240716%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240716T122438Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2e52a676f2a51f4847ca9131366df1ecc04da74c7e9134d073427852b371268223959188cfa667412a6c5445fad4a146d9872ec8acc5ec261ba90d41d16dee76b83b765b125a573969254fee9e08496799bcaa4fc0c052c4a2c0885708e772ff81eb296b9df251bea426d95a8697163c338db8fcdb2d8b4832610da1a18719243f6804647d4576726b770465bb7f005a2ef469bfd2b153138f3187cb921b38443ab7bc077650b5195bdb72bc4ac487548bea282e66cd66edc8850704ab810f9005710bb575adc9c504d2d512ea16601bb0dad783dad6b6045ddd7ee152fe6319466b75261be089ef69115c00168534581662a80ce98b25faef794b716d82b5c0,publicassistance:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F10128%2F5438389%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240716%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240716T122438Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D427536624def5d255e994cf26bafeb615ffc42bcc301744f1b1b9b0b087bc91e3aaff6ac9a2554f46547be0f3d4fab9c200b1eb63cb79a801d15c35956b56d4b2bfef6dbdc1123862ff3f91290a5abb49e03826dfd4e457b8e5f177b24e78eb2004cec73d8d9db8a8d83fe6b089468067380b5dde2bd60a6deffd6ce92377ed871db11ce772c9f1753b3c3c72b780381e4a801e7d30e37aaaafe5d83ce39233a43d1db1c912300aeccc07f30e7a7b48311c6713adf716bc1336cf4a2966348e48313a7d0008107f541b15c3c33e7f1eb3420a22dcceb0c75126e316735d8e9bf1057e142fbf6d023b8cbd51ec87041acbeaa09b7e90722bccbe688c5bf36ec07'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**This notebook is an exercise in the [Pandas](https://www.kaggle.com/learn/pandas) course.  You can reference the tutorial at [this link](https://www.kaggle.com/residentmario/grouping-and-sorting).**

---


# Introduction

In these exercises we'll apply groupwise analysis to our dataset.

Run the code cell below to load the data before running the exercises.

In [ ]:
import pandas as pd

reviews = pd.read_csv("../input/wine-reviews/winemag-data-130k-v2.csv", index_col=0)
pd.set_option("display.max_rows", 5)
print(reviews)

from learntools.core import binder; binder.bind(globals())
from learntools.pandas.grouping_and_sorting import *
print("Setup complete.")

# Exercises

## 1.
Who are the most common wine reviewers in the dataset? Create a `Series` whose index is the `taster_twitter_handle` category from the dataset, and whose values count how many reviews each person wrote.

In [ ]:
reviews_written = reviews['taster_twitter_handle'].value_counts().sort_values(ascending = False)
print(reviews_written)

reviews_written = reviews.groupby('taster_twitter_handle').taster_twitter_handle.count().sort_values(ascending = False)
print(reviews_written)

In [ ]:
# Your code here
reviews_written = reviews.groupby('taster_twitter_handle').taster_twitter_handle.count()
print(reviews_written)

# Check your answer
q1.check()

In [ ]:
#q1.hint()
#q1.solution()

## 2.
What is the best wine I can buy for a given amount of money? Create a `Series` whose index is wine prices and whose values is the maximum number of points a wine costing that much was given in a review. Sort the values by price, ascending (so that `4.0` dollars is at the top and `3300.0` dollars is at the bottom).

In [ ]:
best_rating_per_price = reviews.groupby("price").points.max()
print(best_rating_per_price)

In [ ]:
best_rating_per_price = reviews.groupby("price").points.max()

# Check your answer
q2.check()

In [ ]:
q2.hint()
#q2.solution()

## 3.
What are the minimum and maximum prices for each `variety` of wine? Create a `DataFrame` whose index is the `variety` category from the dataset and whose values are the `min` and `max` values thereof.

In [ ]:
price_extremes = reviews.groupby("variety").price.agg(["min", "max"])
print(price_extremes)

# Check your answer
q3.check()

In [ ]:
#q3.hint()
#q3.solution()

## 4.
What are the most expensive wine varieties? Create a variable `sorted_varieties` containing a copy of the dataframe from the previous question where varieties are sorted in descending order based on minimum price, then on maximum price (to break ties).

In [ ]:
sorted_varieties = price_extremes.sort_values(by=['min', 'max'], ascending = False)

# Check your answer
q4.check()

In [ ]:
#q4.hint()
#q4.solution()

## 5.
Create a `Series` whose index is reviewers and whose values is the average review score given out by that reviewer. Hint: you will need the `taster_name` and `points` columns.

In [ ]:
reviewer_mean_ratings = reviews.groupby("taster_name").points.mean()
print(reviewer_mean_ratings)

In [ ]:
reviewer_mean_ratings = reviews.groupby("taster_name").points.mean()

# Check your answer
q5.check()

In [ ]:
#q5.hint()
#q5.solution()

Are there significant differences in the average scores assigned by the various reviewers? Run the cell below to use the `describe()` method to see a summary of the range of values.

In [ ]:
reviewer_mean_ratings.describe()

## 6.
What combination of countries and varieties are most common? Create a `Series` whose index is a `MultiIndex`of `{country, variety}` pairs. For example, a pinot noir produced in the US should map to `{"US", "Pinot Noir"}`. Sort the values in the `Series` in descending order based on wine count.

In [ ]:
country_variety_counts = reviews.groupby(['country','variety']).size().sort_values(ascending= False)
print(country_variety_counts)
# Check your answer
q6.check()

In [ ]:
# q6.hint()
#q6.solution()

# Keep going

Move on to the [**data types and missing data**](https://www.kaggle.com/residentmario/data-types-and-missing-values).

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/pandas/discussion) to chat with other learners.*